In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Paraíba - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_PB.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Paraíba - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - value,Paraíba - IDH,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Consumo de Cimento (t)
0,2003-1,8.315958,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,118.926160,0.182304,0.656510,3.017292e+07,1.631334e+06,7.291169,2.736342e+07,38.426
1,2003-2,8.309895,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,117.724891,0.185909,0.656949,3.021010e+07,1.632950e+06,7.293438,2.737516e+07,31.672
2,2003-3,8.303831,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,118.501965,0.189593,0.657388,3.024728e+07,1.634566e+06,7.295708,2.738689e+07,28.775
3,2003-4,8.297767,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,119.390679,0.193353,0.657827,3.028445e+07,1.636182e+06,7.297978,2.739862e+07,28.571
4,2003-5,8.291704,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,121.517541,0.197186,0.658266,3.032163e+07,1.637798e+06,7.300247,2.741035e+07,31.613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,294.165653,0.679715,NaN,NaN,NaN,NaN,NaN,95.860
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,293.457511,0.678961,NaN,NaN,NaN,NaN,NaN,90.790
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,292.809622,0.677475,NaN,NaN,NaN,NaN,NaN,94.588
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,290.980478,0.676277,NaN,NaN,NaN,NaN,NaN,89.777


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Paraíba - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - value,Paraíba - IDH,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado
0,-0.800943,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.092830,-2.424081,-2.168844,-1.710694,-2.484291,-2.270808,-2.124240
1,-0.805366,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.140285,-2.392248,-2.133132,-1.692339,-2.436666,-2.234559,-2.091568
2,-0.809789,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.109587,-2.359718,-2.097421,-1.673984,-2.389041,-2.198310,-2.058896
3,-0.814212,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.074479,-2.326517,-2.061709,-1.655629,-2.341417,-2.162061,-2.026224
4,-0.818635,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.990458,-2.292670,-2.025998,-1.637274,-2.293792,-2.125812,-1.993552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.272641,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.885384,0.924040,1.235988,1.087341,-0.170948,0.675345,0.728937
188,1.277530,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.959697,0.934231,1.228601,1.077496,-0.163231,0.658514,0.715410
189,1.282419,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,2.094602,0.957607,1.221214,1.067651,-0.155514,0.641683,0.701882
190,1.287308,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,2.203705,0.987746,1.213827,1.057806,-0.147797,0.624852,0.688355


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      31.891
1      26.089
2      32.066
3      26.351
4      27.348
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Paraíba - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Paraíba - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - value,Paraíba - IDH,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado
0,-0.800943,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.092830,-2.424081,-2.168844,-1.710694,-2.484291,-2.270808,-2.124240
1,-0.805366,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.140285,-2.392248,-2.133132,-1.692339,-2.436666,-2.234559,-2.091568
2,-0.809789,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.109587,-2.359718,-2.097421,-1.673984,-2.389041,-2.198310,-2.058896
3,-0.814212,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.074479,-2.326517,-2.061709,-1.655629,-2.341417,-2.162061,-2.026224
4,-0.818635,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.990458,-2.292670,-2.025998,-1.637274,-2.293792,-2.125812,-1.993552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.300589,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,0.747301,0.928050,1.431308,1.193935,-0.099354,0.924381,1.014355
158,1.292964,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,0.751028,0.933525,1.429336,1.195558,-0.105914,0.923232,1.010769
159,1.285339,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,0.748338,0.939401,1.427365,1.197181,-0.112475,0.922084,1.007184
160,1.277715,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,0.708461,0.946049,1.425393,1.198804,-0.119036,0.920935,1.003598


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      31.891
1      26.089
2      32.066
3      26.351
4      27.348
        ...  
157    58.962
158    73.459
159    59.150
160    64.833
161    63.840
Name: Paraíba - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Paraíba - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - value,Paraíba - IDH,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado
126,0.913576,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.690698,0.725920,0.453223,0.844953,0.848711,1.030555,1.033347
127,0.936878,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.689517,0.735237,0.463389,0.864872,0.830125,1.045296,1.048217
128,0.960180,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.699239,0.744336,0.473555,0.884791,0.811540,1.060038,1.063087
129,0.983482,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.679647,0.753547,0.483721,0.904710,0.792954,1.074779,1.077957
130,1.006783,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.659340,0.762358,0.493887,0.924630,0.774368,1.089520,1.092827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.272641,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.885384,0.924040,1.235988,1.087341,-0.170948,0.675345,0.728937
188,1.277530,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.959697,0.934231,1.228601,1.077496,-0.163231,0.658514,0.715410
189,1.282419,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,2.094602,0.957607,1.221214,1.067651,-0.155514,0.641683,0.701882
190,1.287308,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,2.203705,0.987746,1.213827,1.057806,-0.147797,0.624852,0.688355


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(36, activation='tanh', 
                                 return_sequences=True), 
                                 input_shape=(train_input.shape[1], 
                                              train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(180, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, 
                              reshaped_target, 
                              reshaped_test, 
                              reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1658376212, 1541425160, 3947830751, 881091747, 409538349, 3327410115, 3437800481, 1752910949, 3813432188, 158404059, 228579702, 1141113964, 3623161054, 2794764604, 3247733744, 3053319164, 520564146, 2074211345, 415949560, 815035409, 1204077078, 1633484093, 2637438175, 762392282, 672363379, 129625492, 233412266, 2250818451, 3235359556, 2723904336, 1013479981, 1278875886, 2969353934, 3977057804, 2476506812, 2926497469, 2222492188, 1913987061, 2021430138, 3655120475, 2937790619, 568797607, 3019229671, 1702300601, 1750173095, 2097828472, 1477342394, 548867573, 2925065193, 4090931149]


Step: 0 ___________________________________________
val_loss: 42.380165100097656
winner_seed: 1658376212


Step: 1 ___________________________________________
val_loss: 40.47498321533203
winner_seed: 1541425160


Step: 2 ___________________________________________
val_loss: 45.738136291503906


Step: 3 ___________________________________________
val_loss: 32.705570220947266
winner_seed: 881091747


Step: 4 

In [35]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 3s 228ms/step - loss: 4819.1406 - val_loss: 560.6407
Epoch 2/10000
4/4 [==============================] - 0s 14ms/step - loss: 1076.1443 - val_loss: 840.6661
Epoch 3/10000
4/4 [==============================] - 0s 15ms/step - loss: 503.4993 - val_loss: 909.6307
Epoch 4/10000
4/4 [==============================] - 0s 14ms/step - loss: 933.7274 - val_loss: 408.2657
Epoch 5/10000
4/4 [==============================] - 0s 14ms/step - loss: 853.4339 - val_loss: 393.2149
Epoch 6/10000
4/4 [==============================] - 0s 14ms/step - loss: 584.8445 - val_loss: 304.5594
Epoch 7/10000
4/4 [==============================] - 0s 14ms/step - loss: 281.2363 - val_loss: 234.1679
Epoch 8/10000
4/4 [==============================] - 0s 14ms/step - loss: 257.7119 - val_loss: 280.9939
Epoch 9/10000
4/4 [==============================] - 0s 13ms/step - loss: 274.3597 - val_loss: 307.0297
Epoch 10/10000
4/4 [==============================] - 0s 13ms

4/4 [==============================] - 0s 14ms/step - loss: 79.5108 - val_loss: 82.6285
Epoch 80/10000
4/4 [==============================] - 0s 13ms/step - loss: 48.2208 - val_loss: 140.2521
Epoch 81/10000
4/4 [==============================] - 0s 14ms/step - loss: 103.0345 - val_loss: 97.0932
Epoch 82/10000
4/4 [==============================] - 0s 13ms/step - loss: 75.6584 - val_loss: 78.7436
Epoch 83/10000
4/4 [==============================] - 0s 12ms/step - loss: 50.3139 - val_loss: 88.0279
Epoch 84/10000
4/4 [==============================] - 0s 14ms/step - loss: 43.4500 - val_loss: 68.7941
Epoch 85/10000
4/4 [==============================] - 0s 14ms/step - loss: 49.9067 - val_loss: 65.9178
Epoch 86/10000
4/4 [==============================] - 0s 14ms/step - loss: 37.9043 - val_loss: 65.4428
Epoch 87/10000
4/4 [==============================] - 0s 15ms/step - loss: 48.9802 - val_loss: 102.6593
Epoch 88/10000
4/4 [==============================] - 0s 13ms/step - loss: 73.3760 - 

4/4 [==============================] - 0s 13ms/step - loss: 28.9344 - val_loss: 64.7131
Epoch 159/10000
4/4 [==============================] - 0s 12ms/step - loss: 27.3770 - val_loss: 80.6250
Epoch 160/10000
4/4 [==============================] - 0s 13ms/step - loss: 39.8078 - val_loss: 94.3645
Epoch 161/10000
4/4 [==============================] - 0s 13ms/step - loss: 37.2959 - val_loss: 77.7709
Epoch 162/10000
4/4 [==============================] - 0s 12ms/step - loss: 32.2253 - val_loss: 93.9795
Epoch 163/10000
4/4 [==============================] - 0s 12ms/step - loss: 50.3953 - val_loss: 72.2577
Epoch 164/10000
4/4 [==============================] - 0s 12ms/step - loss: 30.1694 - val_loss: 68.3726
Epoch 165/10000
4/4 [==============================] - 0s 12ms/step - loss: 30.6212 - val_loss: 78.7014
Epoch 166/10000
4/4 [==============================] - 0s 12ms/step - loss: 31.8054 - val_loss: 72.0080
Epoch 167/10000
4/4 [==============================] - 0s 13ms/step - loss: 30.8

Epoch 237/10000
4/4 [==============================] - 0s 13ms/step - loss: 34.0797 - val_loss: 137.4606
Epoch 238/10000
4/4 [==============================] - 0s 13ms/step - loss: 53.6733 - val_loss: 58.5535
Epoch 239/10000
4/4 [==============================] - 0s 13ms/step - loss: 29.5524 - val_loss: 50.7374
Epoch 240/10000
4/4 [==============================] - 0s 12ms/step - loss: 28.6387 - val_loss: 53.5512
Epoch 241/10000
4/4 [==============================] - 0s 12ms/step - loss: 26.4706 - val_loss: 51.2622
Epoch 242/10000
4/4 [==============================] - 0s 13ms/step - loss: 29.0798 - val_loss: 54.9247
Epoch 243/10000
4/4 [==============================] - 0s 13ms/step - loss: 31.4511 - val_loss: 49.3903
Epoch 244/10000
4/4 [==============================] - 0s 12ms/step - loss: 27.3438 - val_loss: 59.8469
Epoch 245/10000
4/4 [==============================] - 0s 13ms/step - loss: 28.3613 - val_loss: 54.4967
Epoch 246/10000
4/4 [==============================] - 0s 12ms/

Epoch 316/10000
4/4 [==============================] - 0s 13ms/step - loss: 21.6500 - val_loss: 64.7249
Epoch 317/10000
4/4 [==============================] - 0s 12ms/step - loss: 18.9157 - val_loss: 65.3745
Epoch 318/10000
4/4 [==============================] - 0s 12ms/step - loss: 20.3377 - val_loss: 75.8353
Epoch 319/10000
4/4 [==============================] - 0s 13ms/step - loss: 24.9526 - val_loss: 65.8186
Epoch 320/10000
4/4 [==============================] - 0s 13ms/step - loss: 18.9433 - val_loss: 71.9703
Epoch 321/10000
4/4 [==============================] - 0s 13ms/step - loss: 21.8755 - val_loss: 66.7439
Epoch 322/10000
4/4 [==============================] - 0s 13ms/step - loss: 21.5741 - val_loss: 66.0177
Epoch 323/10000
4/4 [==============================] - 0s 13ms/step - loss: 21.2124 - val_loss: 62.0606
Epoch 324/10000
4/4 [==============================] - 0s 13ms/step - loss: 20.1022 - val_loss: 67.0646
Epoch 325/10000
4/4 [==============================] - 0s 13ms/s

...

4/4 [==============================] - 0s 13ms/step - loss: 14.9143 - val_loss: 49.4313
Epoch 1417/10000
4/4 [==============================] - 0s 13ms/step - loss: 15.6459 - val_loss: 49.3733
Epoch 1418/10000
4/4 [==============================] - 0s 13ms/step - loss: 15.4949 - val_loss: 55.2486
Epoch 1419/10000
4/4 [==============================] - 0s 13ms/step - loss: 16.8498 - val_loss: 56.4206
Epoch 1420/10000
4/4 [==============================] - 0s 12ms/step - loss: 17.1634 - val_loss: 49.7592
Epoch 1421/10000
4/4 [==============================] - 0s 12ms/step - loss: 15.2313 - val_loss: 52.6347
Epoch 1422/10000
4/4 [==============================] - 0s 12ms/step - loss: 16.9263 - val_loss: 52.2860
Epoch 1423/10000
4/4 [==============================] - 0s 12ms/step - loss: 19.6469 - val_loss: 49.1943
Epoch 1424/10000
4/4 [==============================] - 0s 12ms/step - loss: 16.6988 - val_loss: 53.9678
Epoch 1425/10000
4/4 [==============================] - 0s 12ms/step - l

4/4 [==============================] - 0s 13ms/step - loss: 13.6489 - val_loss: 53.2421
Epoch 1495/10000
4/4 [==============================] - 0s 13ms/step - loss: 14.8898 - val_loss: 55.4450
Epoch 1496/10000
4/4 [==============================] - 0s 14ms/step - loss: 13.9642 - val_loss: 51.3178
Epoch 1497/10000
4/4 [==============================] - 0s 14ms/step - loss: 14.8666 - val_loss: 51.9209
Epoch 1498/10000
4/4 [==============================] - 0s 13ms/step - loss: 15.9088 - val_loss: 60.4585
Epoch 1499/10000
4/4 [==============================] - 0s 13ms/step - loss: 18.8188 - val_loss: 55.1057
Epoch 1500/10000
4/4 [==============================] - 0s 14ms/step - loss: 16.9058 - val_loss: 50.1491
Epoch 1501/10000
4/4 [==============================] - 0s 13ms/step - loss: 12.2378 - val_loss: 50.0386
Epoch 1502/10000
4/4 [==============================] - 0s 13ms/step - loss: 14.1989 - val_loss: 53.1547
Epoch 1503/10000
4/4 [==============================] - 0s 13ms/step - l

4/4 [==============================] - 0s 12ms/step - loss: 21.1192 - val_loss: 66.8992
Epoch 1573/10000
4/4 [==============================] - 0s 13ms/step - loss: 19.8185 - val_loss: 65.8571
Epoch 1574/10000
4/4 [==============================] - 0s 12ms/step - loss: 22.1387 - val_loss: 64.3964
Epoch 1575/10000
4/4 [==============================] - 0s 14ms/step - loss: 23.0401 - val_loss: 64.5212
Epoch 1576/10000
4/4 [==============================] - 0s 13ms/step - loss: 19.6837 - val_loss: 68.8284
Epoch 1577/10000
4/4 [==============================] - 0s 13ms/step - loss: 20.7726 - val_loss: 54.2679
Epoch 1578/10000
4/4 [==============================] - 0s 13ms/step - loss: 14.6584 - val_loss: 54.9774
Epoch 1579/10000
4/4 [==============================] - 0s 13ms/step - loss: 13.3796 - val_loss: 52.2549
Epoch 1580/10000
4/4 [==============================] - 0s 13ms/step - loss: 17.8924 - val_loss: 54.3454
Epoch 1581/10000
4/4 [==============================] - 0s 14ms/step - l

4/4 [==============================] - 0s 12ms/step - loss: 16.7520 - val_loss: 63.2608
Epoch 1651/10000
4/4 [==============================] - 0s 12ms/step - loss: 18.1510 - val_loss: 58.5235
Epoch 1652/10000
4/4 [==============================] - 0s 12ms/step - loss: 15.5421 - val_loss: 57.7469
Epoch 1653/10000
4/4 [==============================] - 0s 13ms/step - loss: 14.8154 - val_loss: 57.2903
Epoch 1654/10000
4/4 [==============================] - 0s 13ms/step - loss: 16.3120 - val_loss: 58.7659
Epoch 1655/10000
4/4 [==============================] - 0s 13ms/step - loss: 14.2454 - val_loss: 68.1946
Epoch 1656/10000
4/4 [==============================] - 0s 13ms/step - loss: 16.1396 - val_loss: 60.1289
Epoch 1657/10000
4/4 [==============================] - 0s 14ms/step - loss: 19.1334 - val_loss: 62.3123
Epoch 1658/10000
4/4 [==============================] - 0s 13ms/step - loss: 17.3193 - val_loss: 59.5662
Epoch 1659/10000
4/4 [==============================] - 0s 13ms/step - l

4/4 [==============================] - 0s 13ms/step - loss: 15.2631 - val_loss: 61.1013
Epoch 1729/10000
4/4 [==============================] - 0s 12ms/step - loss: 14.8839 - val_loss: 59.1895
Epoch 1730/10000
4/4 [==============================] - 0s 13ms/step - loss: 16.4841 - val_loss: 59.3163
Epoch 1731/10000
4/4 [==============================] - 0s 12ms/step - loss: 17.6715 - val_loss: 59.2691
Epoch 1732/10000
4/4 [==============================] - 0s 13ms/step - loss: 18.9999 - val_loss: 51.7902
Epoch 1733/10000
4/4 [==============================] - 0s 12ms/step - loss: 16.5119 - val_loss: 49.8901
Epoch 1734/10000
4/4 [==============================] - 0s 12ms/step - loss: 16.1629 - val_loss: 50.5439
Epoch 1735/10000
4/4 [==============================] - 0s 13ms/step - loss: 13.8698 - val_loss: 54.8028
Epoch 1736/10000
4/4 [==============================] - 0s 15ms/step - loss: 14.8562 - val_loss: 51.2250
Epoch 1736: early stopping


In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,63.796326,63.924026,63.90815,63.899754,63.928856,63.995808,64.132149,64.354576,64.399872,64.328056,64.382858,64.545593,64.427055,64.686317,64.799896,64.707909,64.592827,64.531601,64.791328,64.7938,64.731674,64.743011,64.677773,72.39753,74.563156,79.31649,77.896713,79.426544,82.408958,79.88768
Target,61.721,69.24,70.048,67.172,64.22,63.021,62.639,53.839,56.652,53.408,50.543,64.699,60.219,64.217,57.591,64.307,61.971,57.48,68.053,55.231,53.723,57.457,62.755,48.148,62.19,62.219,61.329,81.988,79.341,76.646
Error,2.075325,5.315971,6.139847,3.272243,0.291145,0.974808,1.493149,10.515575,7.747871,10.920055,13.839859,0.153404,4.208054,0.469315,7.208897,0.400909,2.621826,7.051601,3.261673,9.562801,11.008675,7.286011,1.922771,24.249531,12.373158,17.097488,16.567715,2.561455,3.067955,3.241676


In [38]:
display(mae)
display(mape)

6.563359

0.11400351

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[769.59607]] - Target[737.202]| =  Error: [[32.394043]]; MAPE:[[0.04394188]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[783.88074]] - Target[711.152]| =  Error: [[72.72876]]; MAPE:[[0.10226894]]
1/1 [==============================] - 0s 16ms/step
Ano-5: |Prediction[[473.4995]] - Target[423.713]| =  Error: [[49.7865]]; MAPE:[[0.11750052]]


[array([[32.394043]], dtype=float32),
 array([[72.72876]], dtype=float32),
 array([[49.7865]], dtype=float32)]

51.636433

0.087903775